In [1]:
# https://keras.io/
#!pip install -q keras
import keras

Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
print(x_train.shape, x_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)


In [7]:
print(y_train.shape, y_test.shape)

(50000, 10) (10000, 10)


In [8]:
import numpy as np

x_train = np.append(x_train, x_train,axis=0)
y_train = np.append(y_train, y_train,axis=0)
#print(np.append(x_train, x_train,axis=0).shape)
#print(np.append(y_train, y_train,axis=0).shape)


In [9]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(100000, 32, 32, 3) (10000, 32, 32, 3)
(100000, 10) (10000, 10)


# Data Augmentation

In [10]:

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,    # randomly flip images
        fill_mode='nearest')

In [11]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [12]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [13]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [14]:
#num_filter = 12
num_filter = 32
dropout_rate = 0.2
l = 12

input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
#Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [21]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [22]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [23]:
from keras.callbacks import ModelCheckpoint

In [24]:
#filepath = file_prefix + r".best.hdf5"
filepath = "densenet_tr_02-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [25]:
# Save the trained weights in to .h5 format
model.save_weights("densenet_tr_02-{epoch:02d}-{val_acc:.2f}.hdf5")
print("Saved model to disk")

Saved model to disk


In [26]:
#from google.colab import files

#files.download('DNST_model.h5')

In [27]:
callbacks_list = [checkpoint]

## Call the model with the datagen, augmentation

In [28]:
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 456s 583ms/step - loss: 1.9041 - acc: 0.2816 - val_loss: 2.0515 - val_acc: 0.3246

Epoch 00001: val_acc improved from -inf to 0.32460, saving model to densenet_tr_02-01-0.32.hdf5
Epoch 2/5
782/782 [==============================] - 420s 537ms/step - loss: 1.6836 - acc: 0.3671 - val_loss: 1.7934 - val_acc: 0.3637

Epoch 00002: val_acc improved from 0.32460 to 0.36370, saving model to densenet_tr_02-02-0.36.hdf5
Epoch 3/5
782/782 [==============================] - 420s 537ms/step - loss: 1.5755 - acc: 0.4142 - val_loss: 1.6443 - val_acc: 0.4476

Epoch 00003: val_acc improved from 0.36370 to 0.44760, saving model to densenet_tr_02-03-0.45.hdf5
Epoch 4/5
782/782 [==============================] - 420s 537ms/step - loss: 1.4888 - acc: 0.4510 - val_loss: 1.8398 - val_acc: 0.4425

Epoch 00004: val_acc did not improve from 0.44760
Epoch 5/5
782/782 [==============================] - 420s 537ms/step - loss: 1.4224 - acc: 0.4801 - val_loss: 1.

In [29]:
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 420s 537ms/step - loss: 1.3655 - acc: 0.5014 - val_loss: 1.8340 - val_acc: 0.4279

Epoch 00001: val_acc did not improve from 0.51360
Epoch 2/5
782/782 [==============================] - 420s 537ms/step - loss: 1.3125 - acc: 0.5220 - val_loss: 1.6241 - val_acc: 0.4899

Epoch 00002: val_acc did not improve from 0.51360
Epoch 3/5
782/782 [==============================] - 420s 538ms/step - loss: 1.2743 - acc: 0.5370 - val_loss: 1.4895 - val_acc: 0.5280

Epoch 00003: val_acc improved from 0.51360 to 0.52800, saving model to densenet_tr_02-03-0.53.hdf5
Epoch 4/5
782/782 [==============================] - 420s 537ms/step - loss: 1.2404 - acc: 0.5511 - val_loss: 1.6715 - val_acc: 0.4851

Epoch 00004: val_acc did not improve from 0.52800
Epoch 5/5
782/782 [==============================] - 420s 538ms/step - loss: 1.2110 - acc: 0.5619 - val_loss: 1.7438 - val_acc: 0.5080

Epoch 00005: val_acc did not improve from 0.52800


In [30]:
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/10
782/782 [==============================] - 421s 538ms/step - loss: 1.1814 - acc: 0.5727 - val_loss: 1.7766 - val_acc: 0.5101

Epoch 00001: val_acc did not improve from 0.52800
Epoch 2/10
782/782 [==============================] - 421s 539ms/step - loss: 1.1529 - acc: 0.5857 - val_loss: 1.7739 - val_acc: 0.5182

Epoch 00002: val_acc did not improve from 0.52800
Epoch 3/10
782/782 [==============================] - 420s 538ms/step - loss: 1.1350 - acc: 0.5899 - val_loss: 1.8233 - val_acc: 0.4964

Epoch 00003: val_acc did not improve from 0.52800
Epoch 4/10
782/782 [==============================] - 421s 538ms/step - loss: 1.1145 - acc: 0.6011 - val_loss: 1.7392 - val_acc: 0.5186

Epoch 00004: val_acc did not improve from 0.52800
Epoch 5/10
782/782 [==============================] - 421s 538ms/step - loss: 1.0921 - acc: 0.6086 - val_loss: 1.7300 - val_acc: 0.5129

Epoch 00005: val_acc did not improve from 0.52800
Epoch 6/10
782/782 [==============================] - 421s 538ms/

In [31]:
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
782/782 [==============================] - 421s 538ms/step - loss: 0.9984 - acc: 0.6454 - val_loss: 1.2775 - val_acc: 0.6095

Epoch 00001: val_acc did not improve from 0.62780
Epoch 2/5
782/782 [==============================] - 421s 538ms/step - loss: 0.9862 - acc: 0.6492 - val_loss: 1.5867 - val_acc: 0.5723

Epoch 00002: val_acc did not improve from 0.62780
Epoch 3/5
782/782 [==============================] - 421s 538ms/step - loss: 0.9729 - acc: 0.6540 - val_loss: 1.1079 - val_acc: 0.6511

Epoch 00003: val_acc improved from 0.62780 to 0.65110, saving model to densenet_tr_02-03-0.65.hdf5
Epoch 4/5
782/782 [==============================] - 421s 538ms/step - loss: 0.9615 - acc: 0.6571 - val_loss: 1.2940 - val_acc: 0.6191

Epoch 00004: val_acc did not improve from 0.65110
Epoch 5/5
782/782 [==============================] - 421s 538ms/step - loss: 0.9502 - acc: 0.6608 - val_loss: 1.1984 - val_acc: 0.6408

Epoch 00005: val_acc did not improve from 0.65110


In [ ]:
model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/5
471/782 [=================>............] - ETA: 2:42 - loss: 0.9413 - acc: 0.6661

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5481 - acc: 0.8068 - val_loss: 0.8859 - val_acc: 0.7329

Epoch 00001: val_acc improved from -inf to 0.73290, saving model to densenet-01-0.73.hdf5
Epoch 2/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5417 - acc: 0.8087 - val_loss: 0.7878 - val_acc: 0.7589

Epoch 00002: val_acc improved from 0.73290 to 0.75890, saving model to densenet-02-0.76.hdf5
Epoch 3/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5372 - acc: 0.8082 - val_loss: 0.7391 - val_acc: 0.7770

Epoch 00003: val_acc improved from 0.75890 to 0.77700, saving model to densenet-03-0.78.hdf5
Epoch 4/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.5322 - acc: 0.8104 - val_loss: 0.7139 - val_acc: 0.7774

Epoch 00004: val_acc improved from 0.77700 to 0.77740, saving model to densenet-04-0.78.hdf5
Epoch 5/50
21248/50000 [=

In [ ]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3575 - acc: 0.8739 - val_loss: 0.7500 - val_acc: 0.7998

Epoch 00001: val_acc did not improve from 0.81130
Epoch 2/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3542 - acc: 0.8761 - val_loss: 0.7343 - val_acc: 0.7993

Epoch 00002: val_acc did not improve from 0.81130
Epoch 3/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3531 - acc: 0.8749 - val_loss: 0.6764 - val_acc: 0.8116

Epoch 00003: val_acc improved from 0.81130 to 0.81160, saving model to densenet-03-0.81.hdf5
Epoch 4/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3472 - acc: 0.8772 - val_loss: 0.7236 - val_acc: 0.8018

Epoch 00004: val_acc did not improve from 0.81160
Epoch 5/50
50000/50000 [==============================] - 218s 4ms/step - loss: 0.3487 - acc: 0.8770 - val_loss: 0.7087 - val_acc: 0.8135

Epoch 00005: